In [2]:
!pip install -r requirements.txt

import pandas as pd
import numpy as np
import requests
from pandas import json_normalize
import joblib

pd.set_option('future.no_silent_downcasting', True)

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


## Data Manips

In [ ]:
## Widths for post uber events

force_width = 6

success_width = 30

medic_death_width = 15

drops_forced_width = 15

medic_death_capitalize_window = 25

round_losing_medic_death_window = 12

def manipulate_data(data,log):


    ## INFO ##
    # region Info
    map = data['info']['map']
    title = data['info']['title']
    length = data['length']
    date = data['info']['date']


    log = log
    info = pd.DataFrame({'id': [log], 'length': [length], 'map': [map],'title':[title],'date':[date]})
    info['date'] = pd.to_datetime(info['date'], unit='s')
    # If Too Short Skip
    if length < 200:
        return(f'{log} Too Short. Only {length} seconds long')
    
    # endregion
    
    #
    ## PLAYERS ##
    # region Players 
    players = json_normalize(data['players'])
    players = players.T
    players['stats'] = players[0]
    players.drop(columns=[0], inplace=True)
    players.reset_index(inplace=True)
    players[['steamid', 'metric']] = players['index'].str.extract(r'(\[U:1:\d+\])(?:\.(.*))?')

    # Drop where player steam id is NA

    players = players[players['steamid'].notna()]
    
    players = players.pivot(index = 'steamid', columns='metric', values='stats')

    # Drop useless columns and create ka
    players.rename(columns={'as':'airshots'},inplace=True)

    drop_cols = ['airshots','backstabs','headshots','headshots_hit','lks','ic']

    drop_cols = [col for col in drop_cols if col in players.columns]
    players.drop(drop_cols,axis = 1,inplace = True)

    # Sometimes people dont get kills?>>>?? OR ASSITS????
    for col in ['kills','assists']:
        if col not in players.columns:
            players[col] = pd.Series(dtype = 'int')
    
    players['ka'] = players['kills'] + players['assists']

    # Separate medic stats and player stats
    colnames = players.columns
    colnames = [col for col in colnames if "medi" not in col and "uber" not in col and "drops"]

    medic_stats = players[[col for col in players.columns if col not in colnames]]
    players = players[colnames]

   

    # endregion

    ## INFO ##
    # region Info
    map = data['info']['map']
    title = data['info']['title']
    length = data['length']
    date = data['info']['date']



    log = log
    num_players = players.index.nunique()
    info = pd.DataFrame({'id': [log], 'length': [length], 'map': [map],"num_players": [num_players],'title':[title],'date':[date]})
    info['date'] = pd.to_datetime(info['date'], unit='s')


    # endregion

    ## CLASS STATS ##
    # region Class Stats
    class_stats = pd.DataFrame()
    for i in range(0,len(players)):
        player_class =  pd.DataFrame(players.loc[players.index[i],'class_stats'])
        player_class = player_class[player_class['type'] != 'undefined']
        player_class['steamid'] = players.index[i]
        
        class_stats = pd.concat([class_stats, player_class], ignore_index=True)
        
    # Grab Weapon Stats
    #weapon = class_stats[['weapon','steamid','type']].copy()
    if 'weapon' in class_stats.columns:
        class_stats.drop('weapon',axis = 1,inplace = True)

    # Check if class_stats is null??
    if len(class_stats) >0:
        class_stats = class_stats[['steamid'] + [col for col in class_stats.columns if col != 'steamid']]

        class_stats['total_time'] = class_stats['total_time'].abs()

        time_alive = class_stats.groupby('steamid').agg(time = ("total_time","sum")).reset_index()

        class_stats = class_stats.merge(time_alive)

        class_stats.rename(columns = {'time':'total_time',"total_time":"class_time"},inplace = True)

        # Drop the 'class_stats' column from players DataFrame
        # Need to do this here because class stats needs to be created first

        players.drop('class_stats',axis = 1,inplace = True)

    # endregion

    # ## Weapon Stats ##
    # region Weapon Stats
    # weapon_stats = pd.DataFrame()
    # for num,i in enumerate(weapon.index):
    #     player_weapon = (pd.DataFrame(weapon.loc[i,'weapon'])).T
    #     id = weapon.loc[i,'steamid']
    #     type = weapon.loc[i,'type']
    #     player_weapon['steamid'] = id
    #     player_weapon['type'] = type
    #     weapon_stats = pd.concat([weapon_stats, player_weapon])
        
        
    # weapon_stats = weapon_stats.reset_index()
    # weapon_stats = weapon_stats.pivot(index='steamid', columns='index', values=['kills', 'dmg', 'avg_dmg', 'shots', 'hits'])

    # # Flatten the multi-level columns
    # weapon_stats.columns = [f'{val}_{weapon}' for val, weapon in weapon_stats.columns]

    # # Reset index if needed
    # weapon_stats = weapon_stats.reset_index()


    # endregion

    ## TEAMS ## 
    # region Teams 

    teams = json_normalize(data['teams']).T.reset_index()

    teams[['team','metric']] = teams['index'].str.extract(r'(\w+)(?:\.(.*))?')

    teams.drop(columns=['index'], inplace=True)

    teams = teams.pivot(index='team', columns='metric', values=0)

    teams = teams.replace(0,np.nan)

    teams['drop_rate'] = teams['drops'].div(teams['charges']).astype(float).round(4)

    teams.reset_index(inplace = True)
    # endregion

    ## ROUNDS ##
    # region Rounds
    rounds = json_normalize(data['rounds'])

    rounds.drop('events',axis =1,inplace= True)
    colnames = rounds.columns.str.contains('players')

    rounds = rounds[rounds.columns[~colnames]]

    rounds.drop_duplicates(inplace= True)

    rounds.dropna(subset=['winner'], inplace=True)


    rounds = rounds[rounds['length'] > 40]

    rounds.reset_index(inplace=True)

    rounds.rename(columns={"index":"round"},inplace = True)

    ## ROUND EVENTS ## 
    round_events_temp = rounds['events']

    round_events = pd.DataFrame()
    for i in range(0,len(round_events_temp)):
        round_event =  pd.DataFrame(round_events_temp.loc[i])
        round_event['round'] = i
        round_events = pd.concat([round_events, round_event], ignore_index=True)

    # Change order to have round as first column
    cols = ['round'] + [col for col in round_events.columns if col != 'round']

    round_events = round_events[cols]

    rounds.drop('events',axis =1,inplace= True)


    # Drop Short Rounds

    # endregion

    ## PLAYER ROUNDS ##
    # region Player Rounds
    player_rounds = json_normalize(data['rounds']).drop('events',axis =1)

    colnames = player_rounds.columns.str.contains('players')

    player_rounds = player_rounds[player_rounds.columns[colnames]]

    player_rounds = player_rounds.T

    player_rounds.index = player_rounds.index.str.replace('players.','')

    player_rounds.reset_index(inplace=True)

    player_rounds[['steamid','metric']] = player_rounds['index'].str.extract(r'(\[.*\]).(.*)')

    value_vars = [col for col in player_rounds.columns if col not in ['steamid','index','metric']]

    # Melt into long format
    long_df = player_rounds.melt(
        id_vars=['steamid', 'metric'],
        value_vars=value_vars,
        var_name='round',
        value_name='value'
    )
    long_df = long_df[long_df['steamid'].notna()]
    # Pivot to get the desired format
    player_rounds = long_df.pivot(index = ['steamid','round'], columns='metric', values='value').reset_index()


    # endregion

    ## NAMES ##
    # region Names
    names = pd.Series(data['names'],name = 'name')

    players = players.merge(names, left_index=True, right_index=True, how='left')

    # endregion

    # Healspread
    # region Healspread
    healspread = json_normalize(data['healspread']).T.reset_index()

    healspread.rename(columns={0: 'value'}, inplace=True)
    colnames = healspread['index'].str.split('.')

    healspread['healer'] = colnames.str[0]
    healspread['healed'] = colnames.str[1]

    healspread.drop('index', axis=1, inplace=True)

    healspread = healspread[['healer', 'healed', 'value']]

    #endregion

    ## CLASS KDA ##
    # region Class KDA

    # Class Kills
    class_kills = json_normalize(data['classkills']).T.reset_index()

    class_kills.rename(columns={0: 'value'}, inplace=True)

    colnames = class_kills['index'].str.split('.')

    class_kills['steamid'] = colnames.str[0]
    class_kills['killed'] = colnames.str[1]

    class_kills.drop('index', axis=1, inplace=True)

    class_kills = class_kills.pivot(index='steamid', columns='killed', values='value').reset_index()

    class_kills.replace({np.nan: 0}, inplace=True)

    colnames = class_kills.columns

    colnames = [col + '_kills' for col in colnames if col != 'steamid']

    colnames = ['steamid'] + colnames

    class_kills.columns = colnames

    # Class Deaths
    class_deaths = json_normalize(data['classdeaths']).T.reset_index()

    class_deaths.rename(columns={0: 'value'}, inplace=True)

    colnames = class_deaths['index'].str.split('.')

    class_deaths['steamid'] = colnames.str[0]
    class_deaths['died_to'] = colnames.str[1]

    class_deaths.drop('index', axis=1, inplace=True)

    class_deaths = class_deaths.pivot(index='steamid', columns='died_to', values='value').reset_index()

    class_deaths.replace({np.nan: 0}, inplace=True)

    colnames = class_deaths.columns

    colnames = [col + '_deaths' for col in colnames if col != 'steamid']

    colnames = ['steamid'] + colnames

    class_deaths.columns = colnames

    # Class Assists

    class_assists = json_normalize(data['classkillassists']).T.reset_index()

    class_assists.rename(columns={0: 'value'}, inplace=True)

    colnames = class_assists['index'].str.split('.')

    class_assists['steamid'] = colnames.str[0]
    class_assists['assisted'] = colnames.str[1]

    class_assists.drop('index', axis=1, inplace=True)

    class_assists = class_assists.pivot(index='steamid', columns='assisted', values='value').reset_index()

    class_assists.replace({np.nan: 0}, inplace=True)

    colnames = class_assists.columns

    colnames = [col + '_assists' for col in colnames if col != 'steamid']

    colnames = ['steamid'] + colnames

    class_assists.columns = colnames

    # Merge into KDA

    class_kda = class_kills.merge(class_deaths, on='steamid', how='right')
    class_kda = class_kda.merge(class_assists, on='steamid', how='right')

    # endregion

    ## Primary Class ## 
    # region Primary Class
    class_stats.sort_values(by = ['steamid','total_time'],ascending= False, inplace=True)
    primary_class = class_stats.groupby('steamid').first().reset_index()[['steamid', 'type','total_time']]

    primary_class.rename(columns={'type': 'primary_class','total_time':'primary_class_time'}, inplace=True)

    players = primary_class.merge(players, on='steamid', how='left')

    # endregion

    ## Midfight Conversion ##
    # region Midfight Conversion
    converted = pd.Series([0,0],index = ["Red","Blue"],name = 'count')
    for i in range(0,len(rounds)):
        if rounds.loc[i,'winner'] == rounds.loc[i,'firstcap']:
            if rounds.loc[i,'winner'] == "Blue":
                converted["Blue"] = converted["Blue"] + 1
            else:
                converted["Red"] = converted["Red"] + 1

    firstcaps = rounds['firstcap'].value_counts().replace(0,np.nan)

    midpoint_convert = pd.Series(converted.div(firstcaps), name = "midfight_conversion").reset_index().rename(columns= {"index":"team"})


    teams = teams.merge(midpoint_convert,on = "team",how = "left")

    # endregion

    ## Percent time offclassing ##
    # region Percent time offclassing

    primary_class = class_stats.groupby("steamid").first().replace(0,np.nan)

    primary_class['offclass_time'] = primary_class['total_time'] - primary_class['class_time']

    primary_class['offclass_pct'] =( primary_class['offclass_time'].div(primary_class['total_time'])).astype(float).round(4)

    primary_class.reset_index(inplace=True)

    primary_class = primary_class[['steamid','offclass_time','offclass_pct',"total_time"]]

    players = players.merge(primary_class,on = "steamid",how = "right")

    # endregion

    ## Player Match Averages ##
    # region Player Match Averages

    # Make Team Stats
    teams_temp = teams.copy()
    teams_temp.drop(['firstcaps','score','midfight_conversion','drops','charges','deaths'],axis = 1,inplace = True)
    teams_temp.columns = ["team"] + ["team_" + col for col in teams_temp.columns if col != "team"]

    # Making team_deaths and team_assists bc its bugged
    team_stats = players.groupby("team").agg(
        team_deaths = ("deaths","sum"),
        team_assists = ("assists","sum"),
        team_dt = ('dt',"sum"),
        team_dt_real = ('dt_real','sum'),
        team_dmg_real = ('dmg_real','sum'),
        team_hr = ('hr','sum')
        ).reset_index()
    teams_temp = teams_temp.merge(team_stats)
    teams_temp['team_ka'] = teams_temp['team_kills'] + teams_temp['team_assists']

    # Make player avs
    player_av_temp = players.merge(teams_temp,on = "team").replace(0,np.nan)
    player_av = pd.DataFrame()
    player_av['kill_pct']       = player_av_temp['kills'].div(player_av_temp['team_kills']).astype(float)
    player_av['deaths_pct']     = player_av_temp['deaths'].div(player_av_temp['team_deaths']).astype(float)
    player_av['dmg_pct']        = player_av_temp['dmg'].div(player_av_temp['team_dmg']).astype(float)
    player_av['dmg_real_pct']   = player_av_temp['dmg_real'].div(player_av_temp['team_dmg_real']).astype(float)
    player_av['cpc_pct']        = player_av_temp['cpc'].div(player_av_temp['team_caps']).astype(float)
    player_av['ka_pct']         = player_av_temp['ka'].div(player_av_temp['team_ka']).astype(float)
    player_av['assists_pct']    = player_av_temp['assists'].div(player_av_temp['team_assists']).astype(float)
    player_av['dt_pct']         = player_av_temp['dt'].div(player_av_temp['team_dt']).astype(float)
    player_av['dt_real_pct']    = player_av_temp['dt_real'].div(player_av_temp['team_dt_real']).astype(float)
    player_av['hr_pct']         = player_av_temp['hr'].div(player_av_temp['team_hr']).astype(float)


    # Round and add steamid
    player_av = player_av.round(4)
    player_av['steamid'] = player_av_temp['steamid']

    # Merge
    players = players.merge(player_av,on = "steamid")

    # endregion

    ## Class Stat Averages ##
    # region Class Stats Average
    if len(class_stats) > 0:
        player_teams = players[['steamid','team']]

        class_av_temp = class_stats.merge(player_teams).merge(teams_temp[['team','team_kills','team_dmg','team_deaths','team_assists']])

        class_av_temp = class_av_temp.replace(0,np.nan)

        class_av = pd.DataFrame()

        class_av['kill_pct'] = class_av_temp['kills'].div(class_av_temp['team_kills']).astype(float)
        class_av['assists_pct'] = class_av_temp['assists'].div(class_av_temp['team_assists']).astype(float)
        class_av['dmg_pct'] = class_av_temp['dmg'].div(class_av_temp['team_dmg']).astype(float)
        class_av['deaths_pct'] = class_av_temp['deaths'].div(class_av_temp['team_deaths']).astype(float)


        class_av = class_av.round(4)
        class_av['steamid'] = class_av_temp['steamid']
        class_av['type'] = class_av_temp['type']
        class_av['team'] = class_av_temp['team']


        class_av = class_stats.merge(class_av,on = ['steamid','type'],how = "left")

        class_av = class_av.drop('steamid', axis=1).groupby(['team', 'type']).mean().reset_index().round(4)

        # Add team stats to teams DF

        teams_temp.columns = [col.replace("team_","") for col in teams_temp.columns]

        teams_temp = teams_temp[[col for col in teams_temp.columns if col not in teams.columns] + ['team']]

        teams = teams.merge(teams_temp,how = "left")

    # endregion

    ## PUSH STATS ##
    # region push stats


    ### NOT SURE IF NEEDED #### 

    # # Changing round_events names
    # invert_team = pd.Series(["Red","Blue"],index = ['Blue','Red'])

    # wins = round_events[round_events['type'].isin(["pointcap","round_win"])]

    # wins = wins[wins["round"] == 0].reset_index()

    # winning_team = wins.loc[len(wins) - 2]['team']
    # winning_point = wins.loc[len(wins) - 2]['point']

    # # Decide how to loop throuhg point numbers
    # point_names = {}
    # loop_list = []
    # if winning_point == 1.0:
    #     loop_list = [1.0,2.0,3.0,4.0,5.0]
    # elif winning_point == 5.0:
    #     loop_list = [5.0,4.0,3.0,2.0,1.0]

    # # Change point names based on team winner
    # for i,point in enumerate(loop_list):
    #     s = ""
    #     if i == 0:
    #         team = invert_team[winning_team]
    #         s = f'{team}_Last'
    #     elif i == 1:
    #         team = invert_team[winning_team]
    #         s = f'{team}_Second'
    #     elif i == 2:
    #         s = f'Mid'
    #     elif i == 3:
    #         team = winning_team
    #         s = f'{team}_Second'
    #     elif i == 4:
    #         team = winning_team
    #         s = f'{team}_Last'
    #     point_names[point] = s


    #### NOT SURE IF NEEDED

    point_names = {
        1.0: 'Blue_Last',
        2.0: 'Blue_Second',
        3.0: 'Mid',
        4.0: 'Red_Second',
        5.0: 'Red_Last'
    }


    # Change pointnames
    push = round_events[round_events['type'].isin(["pointcap"])].copy()
    if len(push) == 0:
        push['point'] = pd.Series(dtype = 'float')
    push['point'] = [point_names[point] for point in push['point']]

    # Push Types
    pushes = pd.DataFrame(columns=['count','time'])
    invert_team = pd.Series(["Red", "Blue"], index=['Blue', 'Red'])

    for num in push['round'].unique():
        round_pushes = push[push['round'] == num].reset_index(drop=True)

        for i in range(len(round_pushes) - 1):  # no need to go to last index
            team = round_pushes.loc[i, "team"]

            # Create the row index
            point1 = round_pushes.loc[i, 'point']
            point2 = round_pushes.loc[i + 1, 'point']
            next_team = round_pushes.loc[i + 1, 'team']
            row = f'{next_team}: {point1} - {point2}'

            # Calculate time for the event
            time_for_event = round_pushes.loc[i+1,'time'] - round_pushes.loc[i,'time']

            # Initialize the row if it's not already present
            if row not in pushes.index:
                pushes.loc[row] = {'count': 0,'time':0}
            
            # Add in values
            pushes.loc[row, 'count'] += 1
            pushes.loc[row, 'time'] += time_for_event

    # Avoid Divide by Zero
    pushes = pushes.replace(0,np.nan)
    # Create average time per push type
    pushes['av_time'] = pushes['time'].div(pushes['count']).astype(float).round(4)


    # Transforming Push Types
    colnames = ["push_mid",'takeback_mid','push_last','defend_last','push_second','takeback_enemy_second']
    colnames = colnames + [col + '_time' for col in colnames]
    push_stats = pd.DataFrame(index = ['Red','Blue'],columns = colnames)

    # Loop through and make events readable
    for i,index in enumerate(pushes.index):
        # Split and initialize
        l = (index.split(" "))
        value = pushes.loc[index,'count']
        time = pushes.loc[index,'av_time']
        team = l[0].strip(":")
        point = l[1]
        next_point = l[3]

        # If the point is taken back
        if point == next_point:
            if point == "Mid":
                push_stats.loc[team,"takeback_mid"] = value
                push_stats.loc[team,"takeback_mid_time"] = time

            if "Second" in point:
                if team in point:
                    push_stats.loc[team,"defend_last"] = value
                    push_stats.loc[team,"defend_last_time"] = time
                else:
                    push_stats.loc[team,"takeback_enemy_second"] = value
                    push_stats.loc[team,"takeback_enemy_second_time"] = time
        # If the point is pushed onto
        else:
            if "Second" in point and "Last" in next_point:
                push_stats.loc[team,"push_last"] = value
                push_stats.loc[team,"push_last_time"] = time
            elif "Mid" in point and "Second" in next_point:
                push_stats.loc[team,"push_second"] = value
                push_stats.loc[team,"push_second_time"] = time
            elif "Second" in point and "Mid" in next_point:
                push_stats.loc[team,"push_mid"] = value
                push_stats.loc[team,"push_mid_time"] = time              


    # Calculating push rates
    push_stats['push_last_fails'] = push_stats['defend_last'].values.tolist()[::-1]
    push_stats['push_second_fails'] = push_stats['takeback_mid'].values.tolist()[::-1]


    # Avoid divide by zero
    push_stats = push_stats.replace(0,np.nan)

    push_stats['convert_last_rate'] = push_stats['push_last'].div(
        push_stats['push_last_fails'] + push_stats['push_last']
    ).astype(float).round(4).fillna(1)

    push_stats['convert_second_rate'] = push_stats['push_second'].div(
        push_stats['push_second_fails'] + push_stats['push_second']
    ).astype(float).round(4).fillna(1)

    # Fill nas
    for i in push_stats.index:
        for j in push_stats.columns:
            if pd.isna(push_stats.loc[i,j]):
                push_stats.loc[i,j] = 0
    push_stats = push_stats.reset_index().rename(columns={"index":"team"})
    teams = teams.merge(push_stats, how = 'left')

    # Number of rolls
    num_caps = round_events[round_events['type'] == 'pointcap'].groupby('round').size()

    num_caps = num_caps.reset_index().rename(columns={0:"num_caps"})

    num_caps = rounds.merge(num_caps)[['round','num_caps','winner']]

    rolls = num_caps[num_caps['num_caps'] ==3].groupby('winner').size().reset_index().rename(columns={'winner':"team",0:'rolls'})

    teams = teams.merge(rolls,how= 'left')

    # Avoid divide by zero
    teams = teams.replace(0,np.nan)
    teams['roll_rate'] = (teams['rolls'].div(len(rounds))).astype(float).round(4)


    # endregion

    ## Round Length Stats ## 
    # region round length stats

    # Average win length

    win_length = rounds.groupby('winner').agg(av_win_length = ("length","mean")).reset_index().rename(columns={'winner':"team"})

    teams = teams.merge(win_length,how = 'left')

    # Longest win
    longest_win = rounds.sort_values(by = 'length',ascending= False).groupby('winner').first()['length'].reset_index().rename(columns={'winner':"team",'length':"longest_win"})
    teams = teams.merge(longest_win,how = 'left')

    # Shortest win
    longest_win = rounds.sort_values(by = 'length',ascending= False).groupby('winner').last()['length'].reset_index().rename(columns={'winner':"team",'length':"shortest_win"})
    teams = teams.merge(longest_win,how = 'left')

    # endregion

    ## HROI
    # region HROI

    players = players.replace(0,np.nan)
    players['hroi'] = players['dmg'].div(players['hr'])
    players['hroi_real'] = players['dmg_real'].div(players['hr'])

    # endregion

    ## TEAM WINNER ##
    # region team winner
    # Look at score to see what team won
    if len(teams) > 0:
        team_winner = teams.loc[teams['score'] == teams['score'].max(),'team'].values[0]

        teams['winner'] = 0
        for i in teams['team']:
            if i == team_winner:
                teams.loc[teams['team'] == i, 'winner'] = 1
    else:
        teams['winner'] = pd.Series(dtype = 'int')
                
    # endregion

    # Desired aggregation mapping
    agg_map = {
        "medicstats.advantages_lost": ("medicstats.advantages_lost", "sum"),
        "medicstats.avg_time_before_healing": ("medicstats.avg_time_before_healing", "mean"),
        "medicstats.avg_time_before_using": ("medicstats.avg_time_before_using", "mean"),
        "medicstats.avg_time_to_build": ("medicstats.avg_time_to_build", "mean"),
        "medicstats.avg_uber_length": ("medicstats.avg_uber_length", "mean"),
        "medicstats.biggest_advantage_lost": ("medicstats.biggest_advantage_lost", "max"),
        "medicstats.deaths_with_95_99_uber": ("medicstats.deaths_with_95_99_uber", "sum"),
        "medicstats.deaths_within_20s_after_uber": ("medicstats.deaths_within_20s_after_uber", "sum"),
        "ubers": ("ubers", "sum")
    }

    # Merge and prepare the DataFrame
    merged_df = medic_stats.reset_index().merge(player_teams)

    # Filter agg_map to only include columns that actually exist
    existing_agg_map = {
        k: v for k, v in agg_map.items() if k in merged_df.columns
    }

    # Group and aggregate only the columns that exist
    team_medic_stats = merged_df.groupby('team').agg(**existing_agg_map)
    

    team_medic_stats = team_medic_stats.reset_index()
    team_medic_stats = team_medic_stats.merge((teams[['team','drops']]),how = 'left')

    cols_to_av = ['medicstats.deaths_with_95_99_uber','medicstats.deaths_within_20s_after_uber','drops']

    team_medic_stats = team_medic_stats.replace(0,np.nan)
    for col in cols_to_av:
        if col in team_medic_stats.columns:
            team_medic_stats[col + '_rate'] = team_medic_stats[col].div(team_medic_stats['ubers']).astype(float).round(4)
       
    # endregion

    ## Healspread ##
    # region Healspread By Class
    healspread['steamid'] = healspread['healed']

    healspread = healspread.merge(players[['steamid','primary_class','team']])

    healspread.groupby(['team', 'primary_class']).agg(
        value=('value', 'sum'),
    ).reset_index()

    total_heal = healspread.groupby(['team']).agg(
        total_heal = ('value', 'sum'),
    )

    healspread = healspread.merge(total_heal, on='team', how='left')

    # Avoid divide by zero
    healspread = healspread.replace(0,np.nan)

    healspread['pct_heal'] = healspread['value'].div(healspread['total_heal']).astype(float).round(4)

    healspread.drop(['total_heal','healed','healer','steamid'],axis=1, inplace=True)

    healspread = healspread.groupby(['team', 'primary_class']).agg(
        heals=('value', 'sum'), 
        pct_heal=('pct_heal', 'mean')
    ).reset_index()

    healspread.rename(columns={'value': 'heals','primary_class':'class'}, inplace=True)

    healspread = healspread[['team','class','heals','pct_heal']]

    healspread_red = healspread[healspread['team'] == 'Red'].pivot(index='team', columns='class', values=['heals', 'pct_heal'])

    healspread_red.columns = [f'{col[1]}_{col[0]}' for col in healspread_red.columns if col[0] != 'team']

    healspread_blue = healspread[healspread['team'] == 'Blue'].pivot(index='team', columns='class', values=['heals', 'pct_heal'])

    healspread_blue.columns = [f'{col[1]}_{col[0]}' for col in healspread_blue.columns if col[0] != 'team']

    #healspread_blue = healspread_blue[healspread_red.columns]

    healspread = pd.concat([healspread_red, healspread_blue], axis=0)

    # endregion

    ## Class KDA per Class ##
    # region Class KDA per Class
    class_kda_per_class = class_kda.merge(players[['steamid','primary_class','team']], on='steamid', how='left').groupby(['team', 'primary_class']).sum().reset_index().drop('steamid', axis=1).rename(columns={'primary_class': 'class'})

    # Drop unimportant classes
    offclasses = ['spy','sniper','engineer','heavyweapons','pyro']

    class_kda_per_class = class_kda_per_class[[col for col in class_kda_per_class.columns if
                                                not any(offclass in col for offclass in offclasses)]]

    # Pivot kda by class
    value_cols = [col for col in class_kda_per_class.columns if col not in ['team', 'class']]

    class_kda_red = class_kda_per_class[class_kda_per_class['team'] == 'Red'].pivot(index='team', columns='class', values=value_cols)

    class_kda_red.columns = [f'{col[0]}_as_{col[1]}' for col in class_kda_red.columns if col[0] != 'team']

    class_kda_blue = class_kda_per_class[class_kda_per_class['team'] == 'Blue'].pivot(index='team', columns='class', values=value_cols)

    class_kda_blue.columns = [f'{col[0]}_as_{col[1]}' for col in class_kda_blue.columns if col[0] != 'team']

    #class_kda_blue = class_kda_blue[class_kda_red.columns]

    class_kda_per_class = pd.concat([class_kda_red, class_kda_blue], axis=0)
    # endregion

    ## LAST COMEBACKS ##
    # region Last comebacks
    # Groupby team and bind winner
    last_comebacks = push.groupby('round').agg(
        points = ('point','sum')
    ).merge(rounds[['round','winner']], on='round', how='left')

    comebacks = pd.Series([0,0],index = ['Red','Blue'],name = "comebacks")

    # If winner's last is in round
    for i in range(0,len(last_comebacks)):
        team = last_comebacks.loc[i,'winner']
        s = f'{team}_Second'
        points_captured = last_comebacks.loc[i,'points']

        if s in points_captured:
            comebacks[team] += 1

    comebacks = comebacks.reset_index().rename(columns = {"index":"team"})

    teams = teams.merge(comebacks,how = 'left')

    teams = teams.replace(0,np.nan)

    teams['comeback_rate'] = teams['comebacks'].div(teams['score']).astype(float).round(4)
    # endregion
    
    # endregion

    ## Lead changes ##
    # region Lead Changes

    teams['lead_changes'] = teams['push_mid'] + teams['takeback_mid']
    # endregion

    ## Separate teams ##
    # region Separate teams
    team_stats = teams[['team', 'caps', 'charges', 'dmg', 'drops', 'firstcaps', 'kills',
        'score', 'drop_rate', 'midfight_conversion', 'assists', 'dt', 'dt_real',
        'dmg_real', 'hr', 'ka','deaths','av_win_length', 'longest_win', 'shortest_win',
        'winner']]
    push_stats = teams[['push_mid', 'takeback_mid', 'push_last',
        'defend_last', 'push_second', 'takeback_enemy_second', 'push_mid_time',
        'takeback_mid_time', 'push_last_time', 'defend_last_time',
        'push_second_time', 'takeback_enemy_second_time', 'push_last_fails',
        'push_second_fails', 'convert_last_rate', 'convert_second_rate',
        'rolls', 'roll_rate','comebacks', 'comeback_rate', 'lead_changes']] 
    # endregion

    ## POST UBER EVENTS

    # region POST UBER EVENTS

    charge_stats = round_events[round_events['type'] == 'charge'].reset_index()

    medic_forces = pd.Series([0,0],index = ["Red",'Blue'],name = "forces")

    drops_forced = pd.Series([0,0],index = ["Red",'Blue'],name = "drops_forced")

    successful_ubers = pd.Series([0,0],index = ["Red",'Blue'],name = "successful_ubers")

    medic_deaths_forced = pd.Series([0,0],index = ["Red",'Blue'],name = "medic_deaths_forced")

    for i in range(0,len(charge_stats)):
        team = charge_stats.loc[i,'team']
        time = charge_stats.loc[i,'time']

        # Medic Forces / Forced Drops #

        events = round_events[(round_events['time'] > time) & (round_events['time'] <= time + force_width)]
        events = events[~((events['type'] == 'charge') & (events['team'] == team))]
        
        if 'charge' in events['type'].values:
            medic_forces[team] += 1

        # Successful Uber Pushes #
        events = round_events[(round_events['time'] > time) & (round_events['time'] <= time + success_width)]
        events = events[events['type'] == 'pointcap']

        # Check if any point is capped
        if len(events) != 0:
        # Check if the first cap within the width of the uber is for the ubering team
            first_cap_team = events.groupby('type').first()['team'].values[0]
            if first_cap_team == team:
                successful_ubers[team] += 1
        
        # Medic Deaths Forced
        events = round_events[(round_events['time'] > time) & (round_events['time'] <= time + medic_death_width)]
        events = events[(events['team'] != team)]
        events = events[(events['type'] == "medic_death")]
        # Check if there is an event that matches
        if len(events) != 0:
            medic_deaths_forced[team] += 1
        # Drops Forced
        events = round_events[(round_events['time'] > time) & (round_events['time'] <= time + medic_death_width)]
        events = events[(events['team'] != team)]
        events = events[(events['type'] == "drop")]
        # Check if there is an event that matches
        if len(events) != 0:
            drops_forced[team] += 1

    ## MAKE ALL THE STATS INTO RATES AND BIND THEM IN TO MEDIC STATS!!!!!!!

    uber_stats = pd.concat(
        [medic_forces, drops_forced, successful_ubers, medic_deaths_forced],
        axis=1
    ).reset_index().rename(columns={"index":"team"})

    player_deaths = players.groupby(['team','primary_class']).agg(medic_deaths = ("deaths","sum")).reset_index()
    medic_deaths = player_deaths[player_deaths['primary_class'] == 'medic'].drop('primary_class',axis=1)

    team_medic_stats = team_medic_stats.merge(medic_deaths,how = 'left')

    uber_stats['forced'] = uber_stats['forces'].values[::-1]

    team_medic_stats = team_medic_stats.merge(uber_stats,how = 'left')

    team_medic_stats = team_medic_stats.replace(0,np.nan)

    team_medic_stats['successful_uber_rate'] = team_medic_stats['successful_ubers'].div(team_medic_stats['ubers']).astype(float).round(4)

    team_medic_stats['forced_uber_rate'] = team_medic_stats['forced'].div(team_medic_stats['ubers']).astype(float).round(4)

    invert_medic_deaths_forced = pd.Series(team_medic_stats['medic_deaths_forced'].values[::-1])

    team_medic_stats['forced_medic_death_rate'] = invert_medic_deaths_forced.div(team_medic_stats['medic_deaths']).astype(float).round(4)

    invert_drops_forced = pd.Series(team_medic_stats['drops_forced'].values[::-1])

    team_medic_stats['forced_drop_rate'] = invert_drops_forced.div(team_medic_stats['drops']).astype(float).round(4)
    # endregion

    ## POST MEDIC DEATH EVENTS
    # region POST MEDIC DEATH EVENTS 

    medic_deaths_capitalized = pd.Series([0,0],index = ["Red",'Blue'],name = "medic_deaths_capitalized")

    round_losing_medic_deaths = pd.Series([0,0],index = ["Red",'Blue'],name = "round_losing_medic_deaths")


    medic_death_stats = round_events[round_events['type'].isin(['medic_death','drop'])].reset_index()
    for i in range(0,len(medic_death_stats)):
        team = medic_death_stats.loc[i,'team']
        time = medic_death_stats.loc[i,'time']
        
        # Deaths Capitalized

        events = round_events[(round_events['time'] > time) & (round_events['time'] <= time + medic_death_capitalize_window)]
        events = events[((events['type'] == 'pointcap') & (events['team'] != team))]
        
        if len(events) != 0:
            medic_deaths_capitalized[invert_team[team]] += 1
        
        # Round losing med deaths
        events = round_events[(round_events['time'] > time) & (round_events['time'] <= time + round_losing_medic_death_window)]
        events = events[((events['type'] == 'round_win') & (events['team'] != team))]
        
        if len(events) != 0:
            round_losing_medic_deaths[invert_team[team]] += 1


    #  Binding in to team_medic_stats
    medic_death_stats = pd.concat(
        [medic_deaths_capitalized,round_losing_medic_deaths],axis = 1
    ).reset_index().rename(columns={"index":"team"})

    team_medic_stats = team_medic_stats.merge(medic_death_stats,how = 'left')

    team_medic_stats['round_losing_medic_death_rate'] = team_medic_stats['round_losing_medic_deaths'].div(
        team_medic_stats['medic_deaths']
    ).astype(float).round(4)

    invert_medic_deaths = team_medic_stats['medic_deaths'].values[::-1]

    team_medic_stats['medic_death_capitalization_rate'] = team_medic_stats['medic_deaths_capitalized'].div(
        invert_medic_deaths
    ).astype(float).round(4)

    # endregion

    ## HEAL PER SECOND ##

    players['healps'] = players['heal'].div(players['total_time']).astype(float).round(4)

    ## Suicide Rate ##

    players['suicide_rate'] = players['suicides'].div(players['deaths']).astype(float).round(4)

    ### REMAP DFs WITH ID COLUMN ##

    # Mapping from variable names to actual DataFrames
    df_dict = {
        'info':info,
        'players': players,
        'class_av': class_av,
        'class_stats': class_stats,
        'team_stats':team_stats,
        'class_kda': class_kda,
        'class_kda_per_class': class_kda_per_class,
        'team_medic_stats': team_medic_stats,
        'push_stats': push_stats,
        'medic_stats': medic_stats,
        'healspread': healspread,
        'rounds': rounds,
        'round_events':round_events
    }

    # Apply logic to each DataFrame
    for name, df in df_dict.items():
        df = df.copy()
        df['id'] = info.loc[0, 'id']
        df = df[['id'] + [col for col in df.columns if col != 'id']]
        df_dict[name] = df  # update in the dictionary
        globals()[name] = df  # update actual variable in global namespace
    
    return df_dict
    


## Getting Logs

### Function for Grabbing Log Info

In [ ]:

def request_lan_logs(offset = 0,limit = 100):
    url = f'http://logs.tf/api/v1/log?title= RGL &limit={limit}&offset={offset}'
    response = requests.get(url)
        
    if response.status_code == 200:
        logs = response.json()
    else:
        print("Failed to retrieve data from the URL:", response.status_code)
    
    logs = json_normalize(logs['logs'])
    
    if 'date' in logs.columns:
        logs.sort_values(by='date', ascending=True)
        logs['date'] = pd.to_datetime(logs['date'], unit='s')


    lan_matches = logs[(logs['title'].str.lower().str.contains('rgl')) & (logs['views'])]
    lan_matches = lan_matches[~(lan_matches['title'].str.lower().str.contains(' rgl vs '))]
    lan_matches = lan_matches[~(lan_matches['title'].str.lower().str.contains('vs rgl'))]
    return(lan_matches)

### Parameters to use function
offset_change = 80
n = 2000
limit = 100

start = 0

lan_matches = pd.DataFrame()
for i in range(start,n+1):
    if i % np.floor((n- start) /50) == 0:
        print(i)
    result = request_lan_logs(offset = i * offset_change)
    lan_matches = pd.concat([lan_matches,result])

### Limit the matches to only RGL

In [ ]:
lan_matches = lan_matches[lan_matches['players'] > 11]
lan_matches = lan_matches[lan_matches['players'] < 14]

lan_matches['date'] = pd.to_datetime(lan_matches['date'])

lan_matches = lan_matches[lan_matches['date'] > '2016-07-07']

all_logs = lan_matches.drop_duplicates()

all_logs = all_logs[~all_logs['map'].str.startswith("pl_")]

all_logs = all_logs[~all_logs['map'].str.startswith("pass_")]

### Grab Individual Logs

In [ ]:
import time
saved_logs = []

for i,match in enumerate(all_logs['id'].values):
    time.sleep(1)
    log = match
    url = f'https://logs.tf/api/v1/log/{log}'
    response = requests.get(url)

    
    if response.status_code == 200:
        data = response.json()
        if i % np.floor(len(all_logs) / 150) == 0:
            print(f'{i} / {len(all_logs)}')
    else:
        print("Failed to retrieve data from log:{log}:", response.status_code)
    
    saved_logs.append(data)


    


### Manipulate Logs

In [6]:
saved_logs = joblib.load('data/pkls/rgl_matches_pre_manip.pkl')
all_logs = joblib.load('data/pkls/rgl_log_info.pkl')
all_data = []
error_logs = []
for i in range(0,len(saved_logs)):
    if i % np.floor(len(saved_logs) / 10) == 0:
        print(f'{i} / {len(saved_logs)}')
    try:
        all_data.append(manipulate_data(saved_logs[i], all_logs['id'].values[i]))
    except Exception as e:
        print(f"Skipping index {i} due to error: {e}")
        error_logs.append(saved_logs[i])
        continue        


0 / 2270
227 / 2270
454 / 2270
681 / 2270
908 / 2270
Skipping index 1022 due to error: index 0 is out of bounds for axis 0 with size 0
1135 / 2270
1362 / 2270
1589 / 2270
1816 / 2270
2043 / 2270


### Save Into Csvs

In [7]:
df_dict = {
        'info':pd.DataFrame(),
        'players': pd.DataFrame(),
        'class_av': pd.DataFrame(),
        'class_stats': pd.DataFrame(),
        'team_stats':pd.DataFrame(),
        'class_kda': pd.DataFrame(),
        'class_kda_per_class': pd.DataFrame(),
        'team_medic_stats':pd.DataFrame(),
        'push_stats': pd.DataFrame(),
        'medic_stats': pd.DataFrame(),
        'healspread': pd.DataFrame(),
        'rounds': pd.DataFrame(),
        'round_events':pd.DataFrame()
    }
# Concatenate
for log in all_data:
    # Skip bad ids
    if type(log) == type('streewqoheqoiwehwqoiheowqheqw'):
        continue
    for key in df_dict.keys():
        df = df_dict[key]
        df = pd.concat([df,log[key]])
        df_dict[key] = df

# Save into csv
for key in df_dict.keys():
    df = df_dict[key]
    name = f'data/RGL/{key}.csv'
    df.to_csv(name)

joblib.dump(df_dict,'data/pkls/rgl_df_dict.pkl')

['data/pkls/rgl_df_dict.pkl']

In [8]:
a = joblib.load("../data/pkls/new_data.pkl")

In [18]:
a[719].team_medic_stats

,team,medicstats.advantages_lost,medicstats.avg_time_before_healing,medicstats.avg_time_before_using,medicstats.avg_time_to_build,medicstats.avg_uber_length,medicstats.biggest_advantage_lost,medicstats.deaths_with_95_99_uber,medicstats.deaths_within_20s_after_uber,ubers,...,exchanges_not_initiated,successful_uber_rate,forced_medic_death_rate,forced_drop_rate,medic_deaths_capitalized,round_losing_medic_deaths,round_losing_medic_death_rate,medic_death_capitalization_rate,advantages_lost_per_round,uberspm
0,Blue,NaN,9.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.5,NaN,1,0,0.0,0.5,NaN,NaN
1,Red,NaN,2.2,5.0,61.0,6.1,NaN,NaN,1,1,...,NaN,NaN,NaN,NaN,0,0,0.0,0.0,NaN,0.375


In [5]:
a.drop_duplicates()

,id,title,map,date,views,players
0,3885827,thisismebtw.ifyouevenca.re: wBLED vs RED,pass_ore_a5,2025-07-01 17:47:23,0,8.0
1,3885826,serveme.tf #1493288,koth_product_final,2025-07-01 17:27:51,4,18.0
2,3885825,TF2Center Lobby #1299972,koth_product_final,2025-07-01 17:27:51,19,18.0
3,3885824,serveme.tf #1493292,cp_snakewater_final1,2025-07-01 17:27:36,13,12.0
4,3885823,serveme.tf #1493286,cp_process_f12,2025-07-01 17:22:36,3,14.0
...,...,...,...,...,...,...
95,4322,ShM vs. BiT,,2013-01-19 03:45:09,328,18.0
96,4321,StarKnight on Gully 2,cp_gullywash_final2,2013-01-19 03:29:18,287,12.0
97,4320,StarKnight on Gully 1,cp_gullywash_final1,2013-01-19 03:29:16,238,12.0
98,4319,CNA v Fable Round 2,waste,2013-01-19 03:27:02,448,18.0


In [7]:
a[a['title'].str.lower().str.contains('rgl')]


,id,title,map,date,views,players
89,3795757,RGL PUG Server A: BLU vs RED,cp_metalworks_f5,2025-02-02 09:02:40,219,12.0
96,3795750,RGL PUG Server A: BLU vs RED,cp_sunshine,2025-02-02 08:18:32,144,12.0
34,3785812,RGL PUG Server A: BLU vs RED,cp_sunshine,2025-01-16 02:43:01,25,13.0
77,3785769,RGL PUG Server A: BLU vs RED,cp_sunshine,2025-01-16 01:54:30,18,12.0
88,3785758,RGL PUG Server A: BLU vs RED,cp_process_f12,2025-01-16 01:07:57,43,12.0
...,...,...,...,...,...,...
72,2069830,[PAYLOAD] Combined Logs - RGL Coalplant,RGL Coalplant,2018-07-13 02:42:39,82,14.0
30,1879808,RGL.gg Match Server: .gen vs .gen,pl_badwater_pro_v9,2017-11-13 03:34:53,243,14.0
57,1879781,RGL.gg Match Server: .gen vs .gen,pl_badwater_pro_v9,2017-11-13 03:14:35,226,14.0
7,1859831,RGL.gg Match Server: RED vs PETA,pl_upward,2017-10-16 00:53:40,70,14.0
